In [13]:

outputdebug = False 
def debug(msg):
    if outputdebug:
        print(msg)

class Node():
    def __init__(self, key):
        self.key = key
        self.left = None 
        self.right = None 




class AVLTree():
    def __init__(self, *args):
        self.node = None 
        self.height = -1  
        self.balance = 0; 
        
        if len(args) == 1: 
            for i in args[0]: 
                self.insert(i)
                
    def height(self):
        if self.node: 
            return self.node.height 
        else: 
            return 0 
    
    def is_leaf(self):
        return (self.height == 0) 
    
    def insert(self, key):
        tree = self.node
        
        newnode = Node(key)
        
        if tree == None:
            self.node = newnode 
            self.node.left = AVLTree() 
            self.node.right = AVLTree()
            debug("Inserted key [" + str(key) + "]")
        
        elif key < tree.key: 
            self.node.left.insert(key)
            
        elif key > tree.key: 
            self.node.right.insert(key)
        
        else: 
            debug("Key [" + str(key) + "] already in tree.")
            
        self.rebalance() 
        
    def rebalance(self):
        ''' 
        Rebalance a particular (sub)tree
        ''' 
        # key inserted. Let's check if we're balanced
        self.update_heights(False)
        self.update_balances(False)
        while self.balance < -1 or self.balance > 1: 
            if self.balance > 1:
                if self.node.left.balance < 0:  
                    self.node.left.lrotate() # we're in case II
                    self.update_heights()
                    self.update_balances()
                self.rrotate()
                self.update_heights()
                self.update_balances()
                
            if self.balance < -1:
                if self.node.right.balance > 0:  
                    self.node.right.rrotate() # we're in case III
                    self.update_heights()
                    self.update_balances()
                self.lrotate()
                self.update_heights()
                self.update_balances()


            
    def rrotate(self):
        # Rotate left pivoting on self
        debug ('Rotating ' + str(self.node.key) + ' right') 
        A = self.node 
        B = self.node.left.node 
        T = B.right.node 
        
        self.node = B 
        B.right.node = A 
        A.left.node = T 

    
    def lrotate(self):
        # Rotate left pivoting on self
        debug ('Rotating ' + str(self.node.key) + ' left') 
        A = self.node 
        B = self.node.right.node 
        T = B.left.node 
        
        self.node = B 
        B.left.node = A 
        A.right.node = T 
        
            
    def update_heights(self, recurse=True):
        if not self.node == None: 
            if recurse: 
                if self.node.left != None: 
                    self.node.left.update_heights()
                if self.node.right != None:
                    self.node.right.update_heights()
            
            self.height = max(self.node.left.height,
                              self.node.right.height) + 1 
        else: 
            self.height = -1 
            
    def update_balances(self, recurse=True):
        if not self.node == None: 
            if recurse: 
                if self.node.left != None: 
                    self.node.left.update_balances()
                if self.node.right != None:
                    self.node.right.update_balances()

            self.balance = self.node.left.height - self.node.right.height 
        else: 
            self.balance = 0 

    def delete(self, key):
        # debug("Trying to delete at node: " + str(self.node.key))
        if self.node != None: 
            if self.node.key == key: 
                debug("Deleting ... " + str(key))  
                if self.node.left.node == None and self.node.right.node == None:
                    self.node = None # leaves can be killed at will 
                # if only one subtree, take that 
                elif self.node.left.node == None: 
                    self.node = self.node.right.node
                elif self.node.right.node == None: 
                    self.node = self.node.left.node
                
                # worst-case: both children present. Find logical successor
                else:  
                    replacement = self.logical_successor(self.node)
                    if replacement != None: # sanity check 
                        debug("Found replacement for " + str(key) + " -> " + str(replacement.key))  
                        self.node.key = replacement.key 
                        
                        # replaced. Now delete the key from right child 
                        self.node.right.delete(replacement.key)
                    
                self.rebalance()
                return  
            elif key < self.node.key: 
                self.node.left.delete(key)  
            elif key > self.node.key: 
                self.node.right.delete(key)
                        
            self.rebalance()
        else: 
            return 

    def logical_predecessor(self, node):
        ''' 
        Find the biggest valued node in LEFT child
        ''' 
        node = node.left.node 
        if node != None: 
            while node.right != None:
                if node.right.node == None: 
                    return node 
                else: 
                    node = node.right.node  
        return node 
    
    def logical_successor(self, node):
        ''' 
        Find the smallese valued node in RIGHT child
        ''' 
        node = node.right.node  
        if node != None: # just a sanity check  
            
            while node.left != None:
                debug("LS: traversing: " + str(node.key))
                if node.left.node == None: 
                    return node 
                else: 
                    node = node.left.node  
        return node 

    def check_balanced(self):
        if self == None or self.node == None: 
            return True
        
        # We always need to make sure we are balanced 
        self.update_heights()
        self.update_balances()
        return ((abs(self.balance) < 2) and self.node.left.check_balanced() and self.node.right.check_balanced())  
        
    def inorder_traverse(self):
        if self.node == None:
            return [] 
        
        inlist = [] 
        l = self.node.left.inorder_traverse()
        for i in l: 
            inlist.append(i) 

        inlist.append(self.node.key)

        l = self.node.right.inorder_traverse()
        for i in l: 
            inlist.append(i) 
    
        return inlist 

    def display(self, level=0, pref=''):
        '''
        Display the whole tree. Uses recursive def.
        TODO: create a better display using breadth-first search
        '''        
        self.update_heights()  # Must update heights before balances 
        self.update_balances()
        if(self.node != None): 
            print('-' * level * 2, pref, self.node.key, "[" + str(self.height) + ":" + str(self.balance) + "]", 'L' if self.is_leaf() else ' ')   
            if self.node.left != None: 
                self.node.left.display(level + 1, '<')
            if self.node.left != None:
                self.node.right.display(level + 1, '>')
        



In [18]:
import time
import numpy as np


NUM = 1000
timeSum = []
heightSum = []

for i in range(10):
    t = AVLTree()
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    for num in nums:
        t.insert(num)
    timeSum.append(time.time() - start)
    heightSum.append(t.height)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    print(i+1,"번째 트리 높이: ",t.height)
    
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(randNum)
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()
print(np.mean(timeSum))
print(np.mean(heightSum))

1 번째 트리 생성 시간:  0.031241893768310547
1 번째 트리 높이:  11
1 - 1 번째 탐색, 찾는 숫자 9356469 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 7587395 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 4136533 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 9681214 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 6914171 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  0.031271934509277344
2 번째 트리 높이:  11
2 - 1 번째 탐색, 찾는 숫자 102697 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 2550562 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 1815729 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 9177823 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 5680634 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  0.0312497615814209
3 번째 트리 높이:  11
3 - 1 번째 탐색, 찾는 숫자 1234648 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 5474348 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 5076905 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 1278892 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 7829045 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  0.03836989402770996
4 번째 트리 높이:  11
4 - 1 번째 탐색, 찾는 숫자 2039798 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 2788329 탐색+삭제 시간: 0.0009961128234863281
4 - 3 번째 탐색, 찾는 숫자 9649316 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 7132357 탐색+삭제 

In [22]:
import time
import numpy as np

NUM = 10000
timeSum = []
heightSum = []
for i in range(10):
    t = AVLTree()
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    for num in nums:
        t.insert(num)
    timeSum.append(time.time() - start)
    heightSum.append(t.height)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    print(i+1,"번째 트리 높이: ",t.height)
    
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(randNum)
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()

print(np.mean(timeSum))
print(np.mean(heightSum))

1 번째 트리 생성 시간:  0.31400632858276367
1 번째 트리 높이:  15
1 - 1 번째 탐색, 찾는 숫자 4747837 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 214993 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 1658903 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 2660573 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 9264943 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  0.3372330665588379
2 번째 트리 높이:  15
2 - 1 번째 탐색, 찾는 숫자 5590231 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 689206 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 2839266 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 328659 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 4890137 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  0.3390312194824219
3 번째 트리 높이:  15
3 - 1 번째 탐색, 찾는 숫자 9024991 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 2713531 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 4659567 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 848773 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 8666613 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  0.365276575088501
4 번째 트리 높이:  15
4 - 1 번째 탐색, 찾는 숫자 3245836 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 9192491 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 8625222 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 1961635 탐색+삭제 시간: 0.0
4 - 5 번째 탐색, 찾는 숫자

In [21]:
import time
import numpy as np

NUM = 100000
timeSum = []
heightSum = []

for i in range(10):
    t = AVLTree()
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    for num in nums:
        t.insert(num)
    timeSum.append(time.time() - start)
    heightSum.append(t.height)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    print(i+1,"번째 트리 높이: ",t.height)
    
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(randNum)
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()
print(np.mean(timeSum))
print(np.mean(heightSum))

1 번째 트리 생성 시간:  4.373596906661987
1 번째 트리 높이:  19
1 - 1 번째 탐색, 찾는 숫자 5037428 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 5587984 탐색+삭제 시간: 0.0
1 - 3 번째 탐색, 찾는 숫자 8306169 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 6105133 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 1721280 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  4.893359661102295
2 번째 트리 높이:  19
2 - 1 번째 탐색, 찾는 숫자 2657935 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 9020364 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 1152194 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 5678310 탐색+삭제 시간: 0.0009469985961914062
2 - 5 번째 탐색, 찾는 숫자 7580605 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  5.533925771713257
3 번째 트리 높이:  19
3 - 1 번째 탐색, 찾는 숫자 1031467 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 5452054 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 3228869 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 2234914 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 1048469 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  5.475163698196411
4 번째 트리 높이:  19
4 - 1 번째 탐색, 찾는 숫자 4352073 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 1532966 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 3736611 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 6532944 탐색+삭제 시간: 0.0


In [20]:
import time
import numpy as np

NUM = 1000000
timeSum = []
heightSum = []

for i in range(10):
    t = AVLTree()
    f = open(str(NUM)+"_"+str(i)+".txt",'r')
    nums = []
    lines = f.readlines()
    for line in lines:
        nums.append(int(line))

    start = time.time()
    for num in nums:
        t.insert(num)
    timeSum.append(time.time() - start)
    heightSum.append(t.height)
    print(i+1,"번째 트리 생성 시간: ",time.time() - start)
    print(i+1,"번째 트리 높이: ",t.height)
    
    for j in range(5):
        start = time.time()
        randNum = nums[np.random.randint(NUM)]
        t.delete(randNum)
        print(i+1,"-",j+1,"번째 탐색, 찾는 숫자",randNum,"탐색+삭제 시간:",time.time() - start)
    
    print()
    
    f.close()
print(np.mean(timeSum))
print(np.mean(heightSum))

1 번째 트리 생성 시간:  67.16236186027527
1 번째 트리 높이:  23
1 - 1 번째 탐색, 찾는 숫자 4312843 탐색+삭제 시간: 0.0
1 - 2 번째 탐색, 찾는 숫자 2721476 탐색+삭제 시간: 0.0009520053863525391
1 - 3 번째 탐색, 찾는 숫자 15429 탐색+삭제 시간: 0.0
1 - 4 번째 탐색, 찾는 숫자 5462849 탐색+삭제 시간: 0.0
1 - 5 번째 탐색, 찾는 숫자 5889269 탐색+삭제 시간: 0.0

2 번째 트리 생성 시간:  65.03789377212524
2 번째 트리 높이:  23
2 - 1 번째 탐색, 찾는 숫자 8641116 탐색+삭제 시간: 0.0
2 - 2 번째 탐색, 찾는 숫자 8026611 탐색+삭제 시간: 0.0
2 - 3 번째 탐색, 찾는 숫자 1727523 탐색+삭제 시간: 0.0
2 - 4 번째 탐색, 찾는 숫자 628859 탐색+삭제 시간: 0.0
2 - 5 번째 탐색, 찾는 숫자 1141637 탐색+삭제 시간: 0.0

3 번째 트리 생성 시간:  63.40482568740845
3 번째 트리 높이:  23
3 - 1 번째 탐색, 찾는 숫자 4490935 탐색+삭제 시간: 0.0
3 - 2 번째 탐색, 찾는 숫자 847055 탐색+삭제 시간: 0.0
3 - 3 번째 탐색, 찾는 숫자 7206476 탐색+삭제 시간: 0.0
3 - 4 번째 탐색, 찾는 숫자 4143405 탐색+삭제 시간: 0.0
3 - 5 번째 탐색, 찾는 숫자 3200611 탐색+삭제 시간: 0.0

4 번째 트리 생성 시간:  66.3922905921936
4 번째 트리 높이:  23
4 - 1 번째 탐색, 찾는 숫자 187353 탐색+삭제 시간: 0.0
4 - 2 번째 탐색, 찾는 숫자 467109 탐색+삭제 시간: 0.0
4 - 3 번째 탐색, 찾는 숫자 3536988 탐색+삭제 시간: 0.0
4 - 4 번째 탐색, 찾는 숫자 6506745 탐색+삭제 시간: 0.0
4 - 5 번